# Воведување а вакцини

In [ ]:
import numpy
import networkx
import epyc
import epydemic
import pandas
import mpmath
from networkx import Graph
from typing import Dict, Any, Union, cast
from epydemic import Dynamics, Process, NetworkGenerator

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.dpi'] = 300
import matplotlib.pyplot as plt
import seaborn
matplotlib.style.use('seaborn')
seaborn.set_context("notebook", font_scale=1.75)

In [ ]:
class MonitoredSIR(epydemic.SIR, epydemic.Monitor):
 
    def __init__(self):
        super(MonitoredSIR, self).__init__()
        
    def build(self, params):
        '''Build the observation process.
        
        :param params: the experimental parameters'''
        super(MonitoredSIR, self).build(params)

        # also monitor other compartments
        self.trackNodesInCompartment(epydemic.SIR.SUSCEPTIBLE)
        self.trackNodesInCompartment(epydemic.SIR.REMOVED)

In [ ]:
class PLCNetworkDynamics(epydemic.StochasticDynamics):
    
    # Experimental paramerters
    N = 'N'
    ALPHA = 'alpha'
    CUTOFF = 'cutoff'
    
    def __init__(self, p):
        super(PLCNetworkDynamics, self).__init__(p)

    def makePowerlawWithCutoff(self, alpha, cutoff):
        C = 1.0 / mpmath.polylog(alpha, numpy.exp(-1.0 / cutoff))
        def p(k):
            return (pow((k + 0.0), -alpha) * numpy.exp(-(k + 0.0) / cutoff)) * C
        return p
    
    def generatePLC(self, N, alpha, cutoff, maxdeg=100):
        p = self.makePowerlawWithCutoff(alpha, cutoff)
        rng = numpy.random.default_rng()
        ns = []
        t = 0
        for i in range(N):
            while True:
                k = rng.integers(1, maxdeg)
                if rng.random() < p(k):
                    ns.append(k)
                    t += k
                    break
        while t % 2 != 0:
            i = rng.integers(0, len(ns) - 1)
            t -= ns[i]
            del ns[i]
            while True:
                k = rng.integers(1, maxdeg)
                if rng.random() < p(k):
                    ns.append(k)
                    t += k
                    break
        return networkx.configuration_model(ns, create_using=networkx.Graph())

    def configure(self, params):
        super(PLCNetworkDynamics, self).configure(params)
        
        # build a random powerlaw-with-cutoff network with the given parameters
        N = params[self.N]
        alpha = params[self.ALPHA]
        cutoff = params[self.CUTOFF]
        g = self.generatePLC(N, alpha, cutoff)
        self._network = g

In [ ]:

# network parameters
N = 10000
alpha = 2

# simulation time
T = 1000

# disease dynamic parameters
pInfected = 0.001
pInfect = 0.01
pRemove = 0.002

In [ ]:
dir(PLCNetworkDynamics)

In [ ]:
# set up the experiment
lab = epyc.Lab()
lab[epydemic.SIR.P_INFECTED] = pInfected
lab[epydemic.SIR.P_INFECT] = pInfect
lab[epydemic.SIR.P_REMOVE] = pRemove
lab[PLCNetworkDynamics.N] = N
lab[PLCNetworkDynamics.ALPHA] = alpha
lab[PLCNetworkDynamics.CUTOFF] = numpy.linspace(10, 80,
                                                num=4)
lab[epydemic.Monitor.DELTA] = T / 50

# perform one monitoried epidemic
m = MonitoredSIR()
e = PLCNetworkDynamics(m)
lab.runExperiment(e)

In [ ]:
df = lab.dataframe()
cutoffs = df[PLCNetworkDynamics.CUTOFF].unique()
    
(fig, axs) = plt.subplots(2, 2, sharex=True, sharey=True,
                          figsize=(10, 10))

for (ax, cutoff) in [ (axs[0][0], cutoffs[0]),
                      (axs[0][1], cutoffs[1]),
                      (axs[1][0], cutoffs[2]), 
                      (axs[1][1], cutoffs[3]) ]:
    rc = df[df[PLCNetworkDynamics.CUTOFF] == cutoff]
    timeseries = rc[MonitoredSIR.TIMESERIES].iloc[0]
    ts = timeseries[MonitoredSIR.OBSERVATIONS]
    sss = timeseries[epydemic.SIR.SUSCEPTIBLE]
    iss = timeseries[epydemic.SIR.INFECTED]
    rss = timeseries[epydemic.SIR.REMOVED]

    ax.plot(ts, sss, 'r.', label='suceptible')
    ax.plot(ts, iss, 'g.', label='infected')
    #ax.plot(ts, rss, 'ks', label='removed')

    ax.set_title('$\\kappa = {kappa:.0f}$'.format(kappa=cutoff))
    ax.set_xlim([0, T])
    ax.set_ylim([0, N])
    ax.legend(loc='upper right')

# fine-tune the diagram
plt.suptitle('SIR over powerlaw networks for different cutoffs ($N = {n}, \\alpha={a}$)'.format(n=N, a=alpha))
for y in range(2):
    axs[y][0].set_ylabel('population that is...')
for x in range(2):
    axs[1][x].set_xlabel('$t$')

plt.show()